In [ ]:
import sympy
import symfem
import numpy as np
import matplotlib.pyplot as plt
from sympy import lambdify
import dill
dill.settings["recurse"] = True
from sympy import Rational as R
from tqdm.notebook import tqdm

## Define necessary symbols

In [ ]:
x, y, p, q = sympy.symbols("x y p q")
x1, x2, x3, x4, x5, x6 = sympy.symbols("x1:7")
y1, y2, y3, y4, y5, y6 = sympy.symbols("y1:7")
s = sympy.symbols("s")

## Get shape functions

In [ ]:
monomial_basis = sympy.Matrix([
    1,
    x,
    x**2,
    x**3,
    y,
    y**2,
    y**3,
    x*y,
    x*y**2,
    x**2*y
])

V = sympy.zeros(10, 10)

for i, basis in enumerate(monomial_basis):
    
    V[i, 0] = basis.subs({x: 0, y: 0})
    V[i, 1] = basis.subs({x: 1, y: 0})
    V[i, 2] = basis.subs({x: 0, y: 1})

    V[i, 3] = -basis.subs(y, 1-x).diff(x).subs({x: 1})
    V[i, 4] = -basis.subs(y, 1-x).diff(x).subs({x: 0})

    V[i, 5] = +basis.subs({x: 0, y:s}).diff(s).subs({s: 0})
    V[i, 6] = +basis.subs({x: 0, y:s}).diff(s).subs({s: 1})

    V[i, 7] = +basis.subs({x: s, y:0}).diff(s).subs({s: 0})
    V[i, 8] = +basis.subs({x: s, y:0}).diff(s).subs({s: 1})

    # center of mass
    V[i, 9] = basis.subs({x: R(1,3), y: R(1,3)})
new_basis = V.inv() @ monomial_basis

## Test the result on an example

In [ ]:
phi = np.linspace(0, 0.5, 100)
r = 1

cx = r*np.cos(phi)
cy = r*np.sin(phi)

points = np.array([
    [0.0, 0.0],
    [cx[0], cy[0]],
    [cx[-1], cy[-1]]
])

rm = np.array([
    [0, -1],
    [1, 0]
])

v21 = points[2] - points[0]
v22 = points[2] - points[0]

v31 = points[1] - points[0]
v32 = points[1] - points[0]

# v11 = np.array([+0.5, +1.0])
# v11 /= np.linalg.norm(v11)
# v12 = np.array([-1.0, -0.5])
# v12 /= np.linalg.norm(v12)

# v11 = (rm @ v31).astype(float)
# v11 /= np.linalg.norm(v11)
# v12 = (rm @ v21).astype(float)
# v12 /= np.linalg.norm(v12)

base_angle = np.arccos(v21@v32 / np.linalg.norm(v21) / np.linalg.norm(v31))
v11 = np.array([-np.sin(phi[+0]), np.cos(phi[+0])]) * base_angle * r
v12 = np.array([-np.sin(phi[-1]), np.cos(phi[-1])]) * base_angle * r

In [ ]:
X = points[0, 0]*new_basis[0] + points[1, 0]*new_basis[1] + points[2, 0]*new_basis[2] + \
    v11[0]*new_basis[3] + v12[0]*new_basis[4] + \
    v21[0]*new_basis[5] + v22[0]*new_basis[6] + \
    v31[0]*new_basis[7] + v32[0]*new_basis[8] + \
    points[:3, 0].mean()*new_basis[9]

Y = points[0, 1]*new_basis[0] + points[1, 1]*new_basis[1] + points[2, 1]*new_basis[2] + \
    v11[1]*new_basis[3] + v12[1]*new_basis[4] + \
    v21[1]*new_basis[5] + v22[1]*new_basis[6] + \
    v31[1]*new_basis[7] + v32[1]*new_basis[8] + \
    points[:3, 1].mean()*new_basis[9]


In [ ]:
tt = np.linspace(0, 1, 100)
e1_x = sympy.lambdify([x, y], X)(tt, 1-tt)
e1_y = sympy.lambdify([x, y], Y)(tt, 1-tt)

e2_x = sympy.lambdify([x, y], X)(0, tt)
e2_y = sympy.lambdify([x, y], Y)(0, tt)

e3_x = sympy.lambdify([x, y], X)(tt, 0)
e3_y = sympy.lambdify([x, y], Y)(tt, 0)

## Visualization

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(e1_x, e1_y, ls="--")
ax.plot([cx[0], cx[-1]], [cy[0], cy[-1]], ls="-.")
ax.plot(cx, cy)
ax.legend(["Cubic apprximation", "Linear approximation", "True curve"])

ax.plot(e2_x, e2_y)
ax.plot(e3_x, e3_y)

ax.quiver(points[1, 0], points[1, 1], v11[0], v11[1])
ax.quiver(points[2, 0], points[2, 1], v12[0], v12[1])

ax.quiver(points[0, 0], points[0, 1], v21[0], v21[1])
ax.quiver(points[2, 0], points[2, 1], v22[0], v22[1])

ax.quiver(points[0, 0], points[0, 1], v31[0], v31[1])
ax.quiver(points[1, 0], points[1, 1], v32[0], v32[1])

ax.axis("equal");
fig.tight_layout()

## Check Jacobian

In [ ]:
J = sympy.Matrix([X, Y]).jacobian([x, y])

In [ ]:
t = np.linspace(0.0, 1, 50)
tx, ty = np.meshgrid(t, t, indexing="ij")
mask = (1-tx >= ty)
tx = tx[mask]
ty = ty[mask]

In [ ]:
tz = sympy.lambdify([x, y], J.det())(tx, ty)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(projection="3d"))
ax.plot_trisurf(tx, ty, tz)
fig.tight_layout()
plt.show()

## Define Argyris basis on the element

In [ ]:
J_ = sympy.Matrix([X, Y]).jacobian([x, y])

In [ ]:
x, y, x_1, y_1, x_2, y_2, x_3, y_3 = sympy.symbols("x y x1 y1 x2 y2 x3 y3")

rotation_matrix = sympy.Matrix(
    [
        [+R(0), +R(1)],
        [-R(1), +R(0)]
    ]
)

t1_hat = sympy.Matrix([-R(1), +R(1)]) / sympy.sqrt(2)
t2_hat = sympy.Matrix([+R(0), +R(1)])
t3_hat = sympy.Matrix([+R(1), +R(0)])

n1_hat = rotation_matrix @ t1_hat
n2_hat = rotation_matrix @ t2_hat
n3_hat = rotation_matrix @ t3_hat

G1_hat = n1_hat.row_join(t1_hat).T
G2_hat = n2_hat.row_join(t2_hat).T
G3_hat = n3_hat.row_join(t3_hat).T

l1 = ((x_3 - x_2) ** 2 + (y_3 - y_2) ** 2) ** R(1, 2)
l2 = ((x_3 - x_1) ** 2 + (y_3 - y_1) ** 2) ** R(1, 2)
l3 = ((x_2 - x_1) ** 2 + (y_2 - y_1) ** 2) ** R(1, 2)

t1 = sympy.Matrix([x_3 - x_2, y_3 - y_2]) / l1
t2 = sympy.Matrix([x_3 - x_1, y_3 - y_1]) / l2
t3 = sympy.Matrix([x_2 - x_1, y_2 - y_1]) / l3

n1 = rotation_matrix @ t1
n2 = rotation_matrix @ t2
n3 = rotation_matrix @ t3

tau_1 = sympy.Matrix([t1[0] ** 2, 2 * t1[0] * t1[1], t1[1] ** 2])
tau_2 = sympy.Matrix([t2[0] ** 2, 2 * t2[0] * t2[1], t2[1] ** 2])
tau_3 = sympy.Matrix([t3[0] ** 2, 2 * t3[0] * t3[1], t3[1] ** 2])

J = J_.inv()

THETA = sympy.Matrix(
    [
        [J[0, 0]**2, 2*J[0, 0]*J[1, 0], J[1, 0]**2],
        [J[0, 1]*J[0, 0], J[0, 1]*J[1, 0] + J[0, 0]*J[1, 1], J[1, 0]*J[1, 1]],
        [J[0, 1]**2, 2*J[0, 1]*J[1, 1], J[1, 1]**2],
    ]
)

G1 = n1.row_join(t1).T
G2 = n2.row_join(t2).T
G3 = n3.row_join(t3).T

B1 = G1_hat @ J_.T @ G1.T
B2 = G2_hat @ J_.T @ G2.T
B3 = G3_hat @ J_.T @ G3.T

In [ ]:
kv = {
    x1: points[0, 0],
    x2: points[1, 0],
    x3: points[2, 0],
    y1: points[0, 1],
    y2: points[1, 1],
    y3: points[2, 1],
}

In [ ]:
V_c = sympy.diag(
    1,
    J_.T.subs({x: 0, y: 0}),
    THETA.subs({x: 0, y: 0}).inv(),
    1,
    J_.T.subs({x: 1, y: 0}),
    THETA.subs({x: 1, y: 0}),
    1,
    J_.T.subs({x: 0, y: 1}),
    THETA.subs({x: 0, y: 1}).inv(),

    B1.subs({x: 0.5, y: 0.5}),
    B2.subs({x: 0.0, y: 0.5}),
    B3.subs({x: 0.5, y: 0.5}),
).subs(kv)

In [ ]:
E = np.zeros((21, 24), dtype=int)
for i in range(21):
    for j in range(24):
        if (i <= 18) and (j <= 18) and (i == j):
            E[i][j] = 1
        elif (i == 19) and (j == 20):
            E[i][j] = 1
        elif (i == 20) and (j == 22):
            E[i][j] = 1
E = sympy.Matrix(E)

In [ ]:
D = sympy.Matrix(
    [
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [
            0,
            0,
            0,
            0,
            0,
            0,
            -R(15, 8) / l1,
            -R(7, 16) * t1[0],
            -R(7, 16) * t1[1],
            -l1 / 32 * tau_1[0],
            -l1 / 32 * tau_1[1],
            -l1 / 32 * tau_1[2],
            +R(15, 8) / l1,
            -R(7, 16) * t1[0],
            -R(7, 16) * t1[1],
            +l1 / 32 * tau_1[0],
            +l1 / 32 * tau_1[1],
            +l1 / 32 * tau_1[2],
            0,
            0,
            0,
        ],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [
            -R(15, 8) / l2,
            -R(7, 16) * t2[0],
            -R(7, 16) * t2[1],
            -l2 / 32 * tau_2[0],
            -l2 / 32 * tau_2[1],
            -l2 / 32 * tau_2[2],
            0,
            0,
            0,
            0,
            0,
            0,
            +R(15, 8) / l2,
            -R(7, 16) * t2[0],
            -R(7, 16) * t2[1],
            +l2 / 32 * tau_2[0],
            +l2 / 32 * tau_2[1],
            +l2 / 32 * tau_2[2],
            0,
            0,
            0,
        ],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [
            -R(15, 8) / l3,
            -R(7, 16) * t3[0],
            -R(7, 16) * t3[1],
            -l3 / 32 * tau_3[0],
            -l3 / 32 * tau_3[1],
            -l3 / 32 * tau_3[2],
            +R(15, 8) / l3,
            -R(7, 16) * t3[0],
            -R(7, 16) * t3[1],
            +l3 / 32 * tau_3[0],
            +l3 / 32 * tau_3[1],
            +l3 / 32 * tau_3[2],
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
        ],
    ]
).subs(kv)

In [ ]:
V = E @ V_c @ D
M = V.T

In [ ]:
argyris_basis = dill.load(open("../calculations/argyris_basis", "rb"))

In [ ]:
physical_basis = sympy.lambdify([x, y], M@argyris_basis)

In [ ]:
grid = np.linspace(0, 1, 30)
grid_x, grid_y = np.meshgrid(grid, grid, indexing="ij")
mask = (grid_y <= 1 - grid_x)
reference_x = grid_x[mask]
reference_y = grid_y[mask]

In [ ]:
physical_x = sympy.lambdify([x, y], X.subs(kv))(reference_x, reference_y)
physical_y = sympy.lambdify([x, y], Y.subs(kv))(reference_x, reference_y)

In [ ]:
iv = np.array([
    1, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 0, 0
])

In [ ]:
physical_z = iv@physical_basis(reference_x, reference_y)[:, 0]

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(projection="3d"))
ax.plot_trisurf(physical_x, physical_y, physical_z)
fig.tight_layout()
plt.show()

In [ ]:
v1x, v1y, v2x, v2y = sympy.symbols("v_{1x} v_{1y} v_{2x} v_{2y}")

X = x1*new_basis[0] + x2*new_basis[1] + x3*new_basis[2] + \
    v1x*new_basis[3] + v2x*new_basis[4] + \
    (x2-x1)*new_basis[5] + (x2-x1)*new_basis[6] + \
    (x3-x1)*new_basis[7] + (x2-x1)*new_basis[8] + \
    (x1+x2+x3)/3*new_basis[9]

Y = y1*new_basis[0] + y2*new_basis[1] + y3*new_basis[2] + \
    v1y*new_basis[3] + v2y*new_basis[4] + \
    (y2-y1)*new_basis[5] + (y2-y1)*new_basis[6] + \
    (y3-y1)*new_basis[7] + (y2-y1)*new_basis[8] + \
    (y1+y2+y3)/3*new_basis[9]


In [ ]:
J = sympy.Matrix([X, Y]).jacobian([x, y])

## Numerical integration

### N = 7

In [ ]:
def gauss_quad_7(function):

    a, b = +0.0651301029022, +0.8697397941956
    c, d = +0.3128654960049, +0.6384441885698
    e, f = +0.0486903154253, +0.2603459660790
    g, h = +0.4793080678419, +0.0266736178044
    i, j = +0.0385568804451, +0.0878076287166
    k = -0.0747850222338

    return (
    function.subs({x:a, y:a})*h + \
    function.subs({x:b, y:a})*h + \
    function.subs({x:a, y:b})*h + \
    function.subs({x:c, y:e})*i + \
    function.subs({x:d, y:c})*i + \
    function.subs({x:e, y:d})*i + \
    function.subs({x:d, y:e})*i + \
    function.subs({x:c, y:d})*i + \
    function.subs({x:e, y:c})*i + \
    function.subs({x:f, y:f})*j + \
    function.subs({x:g, y:f})*j + \
    function.subs({x:f, y:g})*j + \
    function.subs({x:1/3, y:1/3})*k)

In [ ]:
kv = {
    x1: points[0, 0],
    x2: points[1, 0],
    x3: points[2, 0],
    y1: points[0, 1],
    y2: points[1, 1],
    y3: points[2, 1],
    v1x: v11[0],
    v1y: v11[1],
    v2x: v21[0],
    v2y: v21[1],
}

In [ ]:
J_inv = sympy.Matrix(
    [
        [+J[1, 1], -J[0, 1]],
        [-J[1, 0], +J[0, 0]]
    ]) / J.det()

In [ ]:
N = len(argyris_basis)

result_b = [[0 for i in range(N)] for j in range(N)]
result_B = [[0 for i in range(N)] for j in range(N)]
result_A = [[0 for i in range(N)] for j in range(N)]

J_simp = J_inv

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    
    first = argyris_basis[idx]
    second = argyris_basis[jdx]

    # -----------------------U------------------------------------------------------------------------------
    u_x = first.diff(x) * J_simp.row(0)[0] + first.diff(y) * J_simp.row(1)[0]
    u_y = first.diff(x) * J_simp.row(0)[1] + first.diff(y) * J_simp.row(1)[1]

    u_xx = u_x.diff(x) * J_simp.row(0)[0] + u_x.diff(y) * J_simp.row(1)[0]
    u_xy = u_x.diff(x) * J_simp.row(0)[1] + u_x.diff(y) * J_simp.row(1)[1]

    u_yy = u_y.diff(x) * J_simp.row(0)[1] + u_y.diff(y) * J_simp.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------V------------------------------------------------------------------------------
    v_x = second.diff(x) * J_simp.row(0)[0] + second.diff(y) * J_simp.row(1)[0]
    v_y = second.diff(x) * J_simp.row(0)[1] + second.diff(y) * J_simp.row(1)[1]

    v_xx = v_x.diff(x) * J_simp.row(0)[0] + v_x.diff(y) * J_simp.row(1)[0]
    v_xy = v_x.diff(x) * J_simp.row(0)[1] + v_x.diff(y) * J_simp.row(1)[1]

    v_yy = v_y.diff(x) * J_simp.row(0)[1] + v_y.diff(y) * J_simp.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------bilinear-form------------------------------------------------------------------
    A = u_xx * v_xx
    B = u_xy * v_xy
    b = u_xx * v_yy
    C = u_yy * v_yy
    # ------------------------------------------------------------------------------------------------------

    # -----------------------integrals----------------------------------------------------------------------
    # sym_int_B = sympy.integrate(A + R(2) * B + C, (y, 0, 1 - x), (x, 0, 1))
    # sym_int_b = sympy.integrate(A + R(2) * b + C, (y, 0, 1 - x), (x, 0, 1))
    # sym_int_A = sympy.integrate(first * second, (y, 0, 1 - x), (x, 0, 1))

    integral_est = gauss_quad_7(sympy.lambdify([x, y], (A + 2*B + C)*J_simp.det()))


    # result_B[idx][jdx] = sym_int_B
    # result_b[idx][jdx] = sym_int_b
    # result_A[idx][jdx] = sym_int_A
# -----------------------------------------------------------------------------------------------------

# result_b = sympy.Matrix(result_b)
# result_B = sympy.Matrix(result_B)
# result_A = sympy.Matrix(result_A)

# result_A = result_A * abs(J.inv().det())
# result_b = result_b * abs(J.inv().det())
# result_B = result_B * abs(J.inv().det())